In [3]:
import json
#!/usr/bin/env python
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
path_to_json = "/Users/hamza.ahmed/Coding Projects/Python Apps Authentications/Snowflake/snowflake_auth.json"

with open(path_to_json, "r") as handler:
    info = json.load(handler)

/Users/hamza.ahmed/Library/Python/3.8/lib/python/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (7.0.0), please install a version that adheres to: 'pyarrow<6.1.0,>=6.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [29]:
def get_col_types(df):
    
    '''
        Helper function to create/modify Snowflake tables; gets the column and dtype pair for each item in the dataframe

        
        args:
            df: dataframe to evaluate
            
    '''
        
    import numpy as np
    
    # get dtypes and convert to df
    ct = df.dtypes.reset_index().rename(columns={0:'col'})
    ct = ct.apply(lambda x: x.astype(str).str.upper()) # case matching as snowflake needs it in uppers
        
    # only considers objects at this point
    # only considers objects and ints at this point
    ct['col'] = np.where(ct['col']=='OBJECT', 'VARCHAR', ct['col'])
    ct['col'] = np.where(ct['col'].str.contains('DATE'), 'DATETIME', ct['col'])
    ct['col'] = np.where(ct['col'].str.contains('INT'), 'NUMERIC', ct['col'])
    ct['col'] = np.where(ct['col'].str.contains('FLOAT'), 'FLOAT', ct['col'])
    
    # get the column dtype pair
    l = []
    for index, row in ct.iterrows():
        l.append(row['index'] + ' ' + row['col'])
    
    string = ', '.join(l) # convert from list to a string object
    
    string = string.strip()
    
    return string

In [60]:
df = pd.read_csv('/Users/hamza.ahmed/Coding Projects/Data/CRKT Data/All_Knife_Data - All_Knives.csv', sep = ",")

# remove spaces in columns name
df.columns = df.columns.str.replace(' ','_')
df.columns = df.columns.str.replace("(","")
df.columns = df.columns.str.replace(")","")
df.columns = df.columns.str.replace("/","")

/var/folders/x_/94tnwtp142gf931j6d20vqk40000gq/T/ipykernel_94715/2289085378.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace("(","")
/var/folders/x_/94tnwtp142gf931j6d20vqk40000gq/T/ipykernel_94715/2289085378.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(")","")


In [61]:
COL_TYPES = get_col_types(df)
COL_TYPES

'COMPANY_NAME VARCHAR, NO_FAMILIES VARCHAR, FAMILY VARCHAR, NAME VARCHAR, PRICE VARCHAR, BLADE_LENGTH_INCHES FLOAT, OVERALL_LENGTH_INCHES VARCHAR, BLADE_THICKNESS_INCHES FLOAT, WIDTH_INCHES FLOAT, BLADE_MATERIAL2 VARCHAR, BLADE_MATERIAL_TYPE VARCHAR, BLADE_MATERIAL VARCHAR, BLADE_GRIND VARCHAR, BLADE_FINISH VARCHAR, HANDLE_MATERIAL2 VARCHAR, HANDLE_MATERIAL_TYPE VARCHAR, HANDLE_MATERIAL VARCHAR, HANDLE_COLORFINISH VARCHAR, PIVOT_ASSEMBLY VARCHAR, LOCKING_MECHANISM VARCHAR, WEIGHT VARCHAR, HANDLE_THICKNESS_INCHES VARCHAR, SHEATH_WEIGHT_OZ FLOAT, CLOSED_LENGTH VARCHAR, BLADE_EDGE VARCHAR, BLADE_SHAPE VARCHAR, BLADE_HARDNESS VARCHAR, MOLLE_COMPATIBLE VARCHAR, BLADE_STYLE VARCHAR, CLIP_TYPE VARCHAR, CLIP_POSITION VARCHAR, SHEATH_TYPE VARCHAR, USE VARCHAR, LANYARD_HOLE VARCHAR, GLASS_BREAKER VARCHAR, BACKSPACER_MATERIAL FLOAT'

In [22]:
Users = info["users"][0]["name"]

Password = info["users"][0]["password"]

In [23]:


Account = 'cp91638.us-central1.gcp'
Warehouse = 'CRKT_DATABASE'
Database="Knives_DB"
Schema="PUBLIC"
Role = "ACCOUNTADMIN"


In [24]:
# set up connection
conn = snowflake.connector.connect(
            account = Account,
            user = Users,
            password = Password,
            warehouse = Warehouse,
            database = Database,
            schema = Schema,
            role = Role)    

# set up cursor
cur = conn.cursor()

In [58]:
def create_table(table, action, col_type, df):
    
    '''
        Function to create/replace and append to tables in Snowflake
        
        args:
            table: name of the table to create/modify
            action: whether do the initial create/replace or appending; key to control logic
            col_type: string with column name associated dtype, each pair separated by a comma; comes from get_col_types() func
            df: dataframe to load
            
        dependencies: function get_col_types(); helper function to get the col and dtypes to create a table
    '''
    
    if action=='create_replace':
    
        # set up execute
        cur.execute(
            """ CREATE OR REPLACE TABLE 
            """ + table +"""(""" + col_type + """)""") 

        #prep to ensure proper case
        df.columns = [col.upper() for col in df.columns]

        # write df to table
        write_pandas(conn, df, table.upper(), quote_identifiers = True)
        
    elif action=='append':
        
        # convert to a string list of tuples
        df = str(list(df.itertuples(index=False, name=None)))
        # get rid of the list elements so it is a string tuple list
        df = df.replace('[','').replace(']','')
        
        # set up execute
        cur.execute(
            """ INSERT INTO """ + table + """
                VALUES """ + df + """
            """)  

In [62]:
create_table('Knives_table', 'create_replace', COL_TYPES, df)

In [127]:
# cs.execute('USE ROLE ACCOUNTADMIN')
# # cs.execute('CREATE DATABASE Knives_DB')
# cs.execute('USE DATABASE Knives_DB')
# cs.execute('USE SCHEMA PUBLIC')

In [31]:
conn = snowflake.connector.connect(
    user=users,
    password=password,
    account='cp91638.us-central1.gcp',
    session_parameters={
        'QUERY_TAG': 'EndOfMonthFinancials',
    }
)

In [34]:
conn.cursor().execute("CREATE WAREHOUSE IF NOT EXISTS tiny_warehouse_mg")
conn.cursor().execute("USE WAREHOUSE tiny_warehouse_mg")

In [35]:
conn.cursor().execute("CREATE DATABASE IF NOT EXISTS testdb")
conn.cursor().execute("USE DATABASE testdb")




In [38]:
conn.cursor().execute("CREATE SCHEMA IF NOT EXISTS testschema")
conn.cursor().execute("USE SCHEMA testschema")



In [39]:
conn.cursor().execute(
    "CREATE OR REPLACE TABLE "
    "test_table(col1 integer, col2 string)")

In [40]:
conn.cursor().execute(
    "INSERT INTO test_table(col1, col2) "
    "VALUES(123, 'test string1'),(456, 'test string2')")

In [42]:

conn.close()